# Huggingface Sagemaker-sdk - Deploy 🤗 Transformers for inference


1. [Introduction](#Introduction)    
    a. [Build a HuggingFace Docker container and push it to Amazon ECR](#Build-a-HuggingFace-Docker-container-and-push-it-to-Amazon-ECR)    
    b. [Deploy one of the 10 000+ Hugging Face Transformers to Amazon SageMaker for Inference](#Deploy-one-of-the-10-000+-Hugging-Face-Transformers-to-Amazon-SageMaker-for-Inference)   


HuggingFace Inference DLCsとAmazon SageMaker Python SDKを使用して、Transformersモデルをデプロイします。    
このNotebookでは10,000以上のHugging Face Transformersモデルが存在するHuggingFace 🤗 [Hub](https://huggingface.co/models)からAmazon SageMakerに直接デプロイして推論します。

_**Note: 2021/08時点では日本語処理ライブラリの追加のため、あらかじめコンテナイメージを作成する必要があります。**_

## API - [SageMaker Hugging Face Inference Toolkit](https://github.com/aws/sagemaker-huggingface-inference-toolkit)


`transformers pipelines`を利用して、`pipelines`の全機能を簡単に利用できるAPIを設計しました。APIは[🤗 Accelerated Inference API](https://api-inference.huggingface.co/docs/python/html/detailed_parameters.html)のAPIを参考にしています。つまり、入力は `inputs` keyで定義する必要があり、サポートされている `pipelines` のパラメータを追加したい場合には `parameters` keyで追加することができます。以下にリクエストの例を示します。

**text-classification request body**
```python
{
	"inputs": "Camera - You are awarded a SiPix Digital Camera! call 09061221066 fromm landline. Delivery within 28 days."
}
```
**question-answering request body**
```python
{
	"inputs": {
		"question": "What is used for inference?",
		"context": "My Name is Philipp and I live in Nuremberg. This model is used with sagemaker for inference."
	}
}
```
**zero-shot classification request body**
```python
{
	"inputs": "Hi, I recently bought a device from your company but it is not working as advertised and I would like to get reimbursed!",
	"parameters": {
		"candidate_labels": [
			"refund",
			"legal",
			"faq"
		]
	}
}
```

## IAM Role
_**Note**: IAMロールに以下の権限があることを確認してください:_

- AmazonSageMakerFullAccess
- AmazonS3FullAccess
- AmazonEC2ContainerRegistryFullAccess

ECRへイメージをpushするために、IAMに`AmazonEC2ContainerRegistryFullAccess`の権限を付与する必要があります。

In [ ]:
!pip install --upgrade pip
!pip install "sagemaker>=2.48.1" "transformers[ja]==4.6.1" "datasets[s3]==1.6.2" --upgrade

## Create an Amazon ECR registry

In [ ]:
import boto3

sess = boto3.Session()

registry_name = 'huggingface-japanese-inference-cpu'
account = boto3.client('sts').get_caller_identity().get('Account')
region = sess.region_name

!aws ecr create-repository --repository-name {registry_name}

## Build a HuggingFace Docker container and push it to Amazon ECR

Dockerfileは[こちら](https://github.com/aws/deep-learning-containers/blob/master/huggingface/pytorch/inference/docker/1.7/py3/Dockerfile.cpu)を一部修正し、使用しています。    

変更点

- 19行目: `TRANSFORMERS_VERSION` → `TRANSFORMERS_VERSION=4.6.1`
- 116行目: `transformers[sentencepiece]` → `transformers[ja]`

サンプルはCPUインスタンス用となっており、GPUインスタンス上で推論したい場合は[こちら](https://github.com/aws/deep-learning-containers/tree/master/huggingface/pytorch/inference/docker/1.7/py3/cu110)を参照してください。

In [ ]:
%%time

image_label = 'v1'
image = f'{account}.dkr.ecr.{region}.amazonaws.com/{registry_name}:{image_label}'

%cd container_cpu
!docker build -t {registry_name}:{image_label} .
!$(aws ecr get-login --no-include-email --region {region})
!docker tag {registry_name}:{image_label} {image}
!docker push {image}
%cd ../

## Deploy one of the 10 000+ Hugging Face Transformers to Amazon SageMaker for Inference

_これは実験的な機能で、エンドポイントが作成された後にモデルが読み込まれるようになっています。これにより、モデルが10GBを超える場合などでエラーが発生する可能性があります_

🤗 HubからSageMakerにモデルを直接デプロイするには、`HuggingFaceModel`の作成時に2つの環境変数を定義する必要があります:

- `HF_MODEL_ID`: SageMakerエンドポイントを作成する際に、[huggingface.co/models](http://huggingface.co/models) から自動的にロードされるモデルIDを定義します。🤗 Hubは10,000以上のモデルを提供しており、この環境変数で利用できます。

- `HF_TASK`: 使用する🤗 Transformersのパイプラインのタスクを定義します。タスクの完全なリストは [ここ](https://huggingface.co/transformers/main_classes/pipelines.html) にあります。

このサンプルでは、🤗 Hubから https://huggingface.co/abhishek/autonlp-japanese-sentiment-59363 をデプロイします。

In [ ]:
from sagemaker.huggingface import HuggingFaceModel
import sagemaker 

role = sagemaker.get_execution_role()

# Hub Model configuration. https://huggingface.co/models
hub = {
  'HF_MODEL_ID':'abhishek/autonlp-japanese-sentiment-59363', # model_id from hf.co/models
  'HF_TASK':'text-classification' # NLP task you want to use for predictions
}

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
    image_uri=image,
    env=hub,
    role=role, # iam role with permissions to create an Endpoint
    #transformers_version="4.6", # transformers version used
    #pytorch_version="1.7", # pytorch version used
    #py_version="py36", # python version of the DLC
)

In [ ]:
# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
   initial_instance_count=1,
   instance_type="ml.m5.xlarge"
)

In [ ]:
# example request, you always need to define "inputs"
data = {
    "inputs": 'ハワイアンの心和む音楽の中、ちょっとシリアスなドラマが展開していきます。音楽の力ってすごいな、って思いました。'
}

In [ ]:
# request
predictor.predict(data)

In [ ]:
# delete endpoint
predictor.delete_endpoint()